***Code pour creer une base de donnée en csv puis la déplacer sur MongoDB***

In [1]:
from dotenv import load_dotenv
import os

# Charger le fichier .env
load_dotenv()

# Accéder aux variables d'environnement
api_key = os.getenv("CLE_GITHUB")

In [2]:
from github import Github
import base64
import random
import csv
from markdown import markdown
from bs4 import BeautifulSoup

# Remplacez par votre token d'accès personnel
TOKEN = api_key

# Connexion à l'API GitHub
g = Github(TOKEN)

# Fonction pour récupérer des dépôts aléatoires
def get_random_repositories(github, num_repos=4500):
    """Récupère un certain nombre de dépôts publics aléatoires distincts."""
    repositories = []
    seen_repos = set()  # Ensemble pour éviter les doublons
    search_query = "stars:>1"  # Rechercher des dépôts avec au moins 1 étoile

    for repo in github.search_repositories(query=search_query):
        # Utiliser un identifiant unique pour vérifier les doublons (par exemple, le nom complet)
        repo_identifier = repo.full_name  # Exemple : "owner/repository_name"

        if repo_identifier not in seen_repos:
            repositories.append(repo)
            seen_repos.add(repo_identifier)

        # Arrêter la boucle si on a récupéré suffisamment de dépôts
        if len(repositories) >= num_repos:
            break

    return repositories

# Fonction pour convertir le Markdown en texte brut
def markdown_to_text(markdown_content):
    # Convertir le Markdown en HTML
    html_content = markdown(markdown_content)
    # Extraire le texte brut du HTML
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text()

# Fonction pour supprimer les sauts de ligne multiples
def remove_multiple_newlines(text):
    # Remplace plusieurs sauts de ligne consécutifs par un seul saut de ligne
    return "\n".join(line for line in text.splitlines() if line.strip())

# Fonction pour extraire les informations des dépôts
def extract_repo_data(repo):
    """Extrait les informations d'un dépôt donné."""
    try:
        readme = repo.get_readme()
        readme_content = base64.b64decode(readme.content).decode("utf-8")
    except:
        readme_content = "README non disponible"

    last_commit = repo.get_commits()[0]    
    return {
        "Nom du dépôt": repo.name,
        "Propriétaire": repo.owner.login,
        "Topics":repo.get_topics(),
        "Date de création": repo.created_at,
        "Url": repo.html_url,
        "Date du dernier commit": last_commit.commit.author.date if last_commit else "N/A",
        "Langage principal": repo.language,
        "Nombre d'étoiles": repo.stargazers_count,
        "Description": repo.description or "Pas de description",
        "README": remove_multiple_newlines(markdown_to_text(readme_content))  # Limiter à 1000 caractères
    }

# Récupérer des dépôts aléatoires
num_repos = 4500  # Nombre de dépôts à collecter
random_repositories = get_random_repositories(g, num_repos)

# Extraire les informations de chaque dépôt
data = []
i = 0
for repo in random_repositories:
    repo_data = extract_repo_data(repo)
    data.append(repo_data)
    i+=1
    print(f"Dépôt analysé {i}: {repo_data['Nom du dépôt']}")

# Sauvegarder les données dans un fichier CSV
output_file = "repositories.csv"
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()
    writer.writerows(data)

print(f"Données des dépôts sauvegardées dans le fichier : {output_file}")


Request GET /search/repositories?q=stars%3A%3E1&page=14 failed with 403: Forbidden
Setting next backoff to 60s
Request GET /search/repositories?q=stars%3A%3E1&page=28 failed with 403: Forbidden
Setting next backoff to 60s


Dépôt analysé : freeCodeCamp
Dépôt analysé : free-programming-books
Dépôt analysé : awesome
Dépôt analysé : build-your-own-x
Dépôt analysé : public-apis
Dépôt analysé : coding-interview-university
Dépôt analysé : developer-roadmap
Dépôt analysé : system-design-primer
Dépôt analysé : 996.ICU
Dépôt analysé : react
Dépôt analysé : awesome-python
Dépôt analysé : awesome-selfhosted
Dépôt analysé : project-based-learning
Dépôt analysé : vue
Dépôt analysé : Python
Dépôt analysé : javascript-algorithms
Dépôt analysé : tensorflow
Dépôt analysé : linux
Dépôt analysé : You-Dont-Know-JS
Dépôt analysé : CS-Notes
Dépôt analysé : ohmyzsh
Dépôt analysé : computer-science
Dépôt analysé : bootstrap
Dépôt analysé : AutoGPT
Dépôt analysé : flutter
Dépôt analysé : vscode
Dépôt analysé : gitignore
Dépôt analysé : Python-100-Days
Dépôt analysé : the-book-of-secret-knowledge
Dépôt analysé : the-art-of-command-line
Dépôt analysé : JavaGuide
Dépôt analysé : stable-diffusion-webui
Dépôt analysé : javascript
Dépô

In [12]:
import csv
import pandas as pd

# Chemin vers le fichier CSV
csv_file = "bddtest/repositories.csv"

# Lecture et affichage des données
i=1
with open(csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        if(i):
            print(row)
            i=0


['topic\tname\towner\towner_type\tfull_name\tdescription\tog_image\tlicense\tis_archived\tis_forked\tsize\tlanguage\ttags\topen_issues\tforks\tstars\twatchers\thas_wiki\thas_pages\thas_sponsorship\tcreated_at\tupdated_at']


In [14]:
from pymongo import MongoClient
import csv

# Chemin vers le fichier CSV
csv_file = "bddtest/repositories2.csv"
csv.field_size_limit(1000000)

# Se connecter au serveur MongoDB
client = MongoClient("mongodb://127.0.0.1:27017")

# Créer ou accéder à une base de données
db = client["my_database"]

# Créer une collection et insérer un document
collection = db["train_set"]


with open(csv_file, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)  # Utiliser DictReader pour lire les lignes comme des dictionnaires
    for row in reader:
        # Insérer chaque ligne du CSV comme un document dans MongoDB
        collection.insert_one(row)

print("Base de données et collection créées avec succès !")


InvalidDocument: documents must have only string keys, key was None

In [18]:
import pandas as pd
from pymongo import MongoClient

# Charger le fichier CSV avec pandas
csv_file = "bddtest/repositories3.csv"
df = pd.read_csv(csv_file, sep="\t")  # Lire le fichier CSV avec le séparateur tabulation

# Se connecter au serveur MongoDB
client = MongoClient("mongodb://127.0.0.1:27017")

# Créer ou accéder à une base de données
db = client["my_database"]

# Créer une collection
collection = db["train_set"]

# Convertir le DataFrame en une liste de dictionnaires et insérer dans MongoDB
collection.insert_many(df.to_dict(orient="records"))

print("Base de données et collection créées avec succès !")


Base de données et collection créées avec succès !


In [33]:
import pandas as pd
import requests
import base64
from github import Github
import random
import csv
from markdown import markdown
from bs4 import BeautifulSoup
from tqdm import tqdm

HEADERS = {"Authorization": f"token {api_key}"}

def markdown_to_text(markdown_content):
    # Convertir le Markdown en HTML
    html_content = markdown(markdown_content)
    # Extraire le texte brut du HTML
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text()

# Fonction pour supprimer les sauts de ligne multiples
def remove_multiple_newlines(text):
    # Remplace plusieurs sauts de ligne consécutifs par un seul saut de ligne
    return "\n".join(line for line in text.splitlines() if line.strip())


def get_readme_content(readme_url):
    """Télécharge et retourne le contenu brut du README."""
    if not readme_url:
        return "README non disponible"

    response = requests.get(readme_url, headers=HEADERS)
    
    if response.status_code == 200:
        return markdown_to_text(response.text)  # Convertir Markdown en texte brut
    else:
        print(f"❌ Erreur {response.status_code} lors du téléchargement du README")
        return "README non disponible"

def get_readme_url(owner, repo):
    """Récupère l'URL brute du README d'un dépôt GitHub."""
    readme_url = find_readme_url(owner, repo)  # Déjà une URL de téléchargement
    
    if readme_url:
        print(f"✅ README trouvé : {readme_url}")
        return readme_url
    else:
        print(f"❌ Aucun README trouvé pour {owner}/{repo}")
        return None
    
def find_readme_url(owner, repo):
    """Cherche récursivement le fichier README dans le dépôt GitHub."""
    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents"
    
    def search_readme(url):
        """Fonction récursive pour chercher un fichier README dans le dépôt."""
        response = requests.get(url, headers=HEADERS)
        
        if response.status_code != 200:
            return None
        
        files = response.json()
        
        for file in files:
            # Vérifier si c'est un README (quel que soit son format)
            if "readme" in file["name"].lower():
                return file["download_url"]
            
            # Si c'est un dossier, continuer la recherche à l'intérieur
            if file["type"] == "dir":
                sub_dir_url = file["url"]
                found_readme = search_readme(sub_dir_url)  # Appel récursif
                if found_readme:
                    return found_readme
        
        return None  # Aucun README trouvé
    
    return search_readme(api_url)  # Lancer la recherche depuis la racine du dépôt



df = pd.read_csv("bddtest/repositories.csv", sep="\t")
df.head()

colonnes = [
    "Nom du dépôt",
    "Propriétaire",
    "Topics",
    "Date de création",
    "Url",
    "Date du dernier commit",
    "Langage principal",
    "Nombre d'étoiles",
    "Description",
    "README"
]

#topic	name	owner	owner_type	full_name	description	og_image	license	is_archived	is_forked	size	language	tags	open_issues	forks	stars	watchers	has_wiki	has_pages	has_sponsorship	created_at	updated_at


df_test = pd.DataFrame(columns=colonnes)


df_test["Topics"] = df["topic"]
df_test["Nom du dépôt"] = df["name"]
df_test["Propriétaire"] = df["owner"]
df_test["Date de création"] = df["created_at"]
df_test["Date du dernier commit"] = df["updated_at"]
df_test["Langage principal"] = df["language"]
df_test["Nombre d'étoiles"] = df["stars"]
df_test["Description"] = df["description"]


for i in tqdm(df_test.index, desc="Traitement des dépôts"):
    owner = df_test.loc[i, "Propriétaire"]
    repo = df_test.loc[i, "Nom du dépôt"]
    df_test.loc[i, "Url"] = f"https://api.github.com/repos/{owner}/{repo}"
    readme_url = get_readme_url(owner, repo)
    if readme_url != None:
        response = requests.get(readme_url, headers=HEADERS)
        res = remove_multiple_newlines(markdown_to_text(response.text))
        df_test.loc[i, "README"] = res
    else:
        df_test.loc[i, "README"] = "README non disponible"

df_test.to_csv("bddtest/repositories100.csv", sep="\t", index=False)
print("Fichier CSV sauvegardé avec succès !")
print(df_test.head())

Traitement des dépôts:   0%|          | 0/40632 [00:00<?, ?it/s]

✅ README trouvé : https://raw.githubusercontent.com/mrdoob/three.js/dev/README.md


Traitement des dépôts:   0%|          | 1/40632 [00:02<26:34:44,  2.35s/it]

✅ README trouvé : https://raw.githubusercontent.com/libgdx/libgdx/master/README.md


Traitement des dépôts:   0%|          | 2/40632 [00:06<35:24:55,  3.14s/it]

✅ README trouvé : https://raw.githubusercontent.com/pmndrs/react-three-fiber/master/.changeset/README.md


Traitement des dépôts:   0%|          | 3/40632 [00:11<43:41:17,  3.87s/it]


KeyboardInterrupt: 

In [8]:
import requests
import base64

def get_readme_url(owner, repo):
    """Récupère l'URL brute du README d'un dépôt GitHub."""
    api_url = f"https://api.github.com/repos/{owner}/{repo}/readme"
    headers = {"Accept": "application/vnd.github.v3+json"}  # Spécifier la version de l'API
    
    response = requests.get(api_url, headers=headers)

    if response.status_code == 200:
        readme_data = response.json()
        return readme_data["download_url"]  # URL brute du README.md
    else:
        return f"Erreur {response.status_code}: Impossible de récupérer le README."

# Exemple d'utilisation
repo_url = get_readme_url("mrdoob", "three.js")
HEADERS = {"Authorization": f"token {api_key}"}
response = requests.get(repo_url, headers=HEADERS)
#readme_content = base64.b64decode(response.content).decode("utf-8")
print(response.content)


b"# three.js\n\n[![NPM Package][npm]][npm-url]\n[![Build Size][build-size]][build-size-url]\n[![NPM Downloads][npm-downloads]][npmtrends-url]\n[![DeepScan][deepscan]][deepscan-url]\n[![Discord][discord]][discord-url]\n\n#### JavaScript 3D library\n\nThe aim of the project is to create an easy-to-use, lightweight, cross-browser, general-purpose 3D library. The current builds only include WebGL and WebGPU renderers but SVG and CSS3D renderers are also available as addons.\n\n[Examples](https://threejs.org/examples/) &mdash;\n[Docs](https://threejs.org/docs/) &mdash;\n[Manual](https://threejs.org/manual/) &mdash;\n[Wiki](https://github.com/mrdoob/three.js/wiki) &mdash;\n[Migrating](https://github.com/mrdoob/three.js/wiki/Migration-Guide) &mdash;\n[Questions](https://stackoverflow.com/questions/tagged/three.js) &mdash;\n[Forum](https://discourse.threejs.org/) &mdash;\n[Discord](https://discord.gg/56GBJwAnUS)\n\n### Usage\n\nThis code creates a scene, a camera, and a geometric cube, and it 

In [10]:
import numpy as np
tab = [0.38,0.24,0.1,0.1,0.1,0.05,0.03]
res=0
for i in tab:
    res+=i*np.log2(i)
print(-res)

2.3890290262608733


In [9]:
print(0.38+0.48+0.4*3+0.2+0.15)

2.41


In [17]:
import pandas as pd
df = pd.read_csv("bddtest/repositories2.csv", sep="\t")
df2 = pd.read_csv("bddtest/repositories.csv", sep="\t")

# Supprimer la colonne 'topics'
df['Description'] = df2['description']
df.to_csv("bddtest/repositories3.csv", sep="\t", index=False)
print(df)

             Nom du dépôt       Propriétaire Topics      Date de création  \
0                three.js             mrdoob     3d  2010-03-23T18:58:01Z   
1                  libgdx             libgdx     3d  2012-08-10T19:34:38Z   
2       react-three-fiber             pmndrs     3d  2019-02-25T14:31:51Z   
3              Babylon.js          BabylonJS     3d  2013-06-27T20:40:42Z   
4                  aframe           aframevr     3d  2015-09-15T02:04:40Z   
...                   ...                ...    ...                   ...   
40627           sepa_king          salesking    xml  2013-08-10T05:48:23Z   
40628             Sitemap  ThePixelDeveloper    xml  2013-02-02T19:50:31Z   
40629         open-bibles            seven1m    xml  2015-05-07T00:56:19Z   
40630  fedramp-automation                GSA    xml  2019-11-25T16:52:33Z   
40631           soupsieve       facelessuser    xml  2018-12-07T07:42:56Z   

                                                     Url  \
0           htt

In [29]:
import requests

HEADERS = {"Authorization": f"token {api_key}"}

def get_readme_url(owner, repo):
    """Récupère directement l'URL brute du README d'un dépôt GitHub."""
    readme_url = find_readme_url(owner, repo)  # Déjà une URL de téléchargement
    
    if readme_url:
        print(f"✅ README trouvé : {readme_url}")
        return readme_url
    else:
        print(f"❌ Aucun README trouvé pour {owner}/{repo}")
        return None


def find_readme_url(owner, repo):
    """Cherche récursivement le fichier README dans le dépôt GitHub."""
    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents"
    
    def search_readme(url):
        """Fonction récursive pour chercher un fichier README dans le dépôt."""
        response = requests.get(url, headers=HEADERS)
        
        if response.status_code != 200:
            return None
        
        files = response.json()
        
        for file in files:
            # Vérifier si c'est un README (quel que soit son format)
            if "readme" in file["name"].lower():
                return file["download_url"]
            
            # Si c'est un dossier, continuer la recherche à l'intérieur
            if file["type"] == "dir":
                sub_dir_url = file["url"]
                found_readme = search_readme(sub_dir_url)  # Appel récursif
                if found_readme:
                    return found_readme
        
        return None  # Aucun README trouvé
    
    return search_readme(api_url)  # Lancer la recherche depuis la racine du dépôt

#test = find_readme_url("mrdoob", "three.js")



    
test = get_readme_url("mrdoob", "three.js")

print(test)

✅ README trouvé : https://raw.githubusercontent.com/mrdoob/three.js/dev/README.md
https://raw.githubusercontent.com/mrdoob/three.js/dev/README.md


In [32]:
import requests
from markdown import markdown
from bs4 import BeautifulSoup

HEADERS = {"Authorization": f"token {api_key}"}

def markdown_to_text(markdown_content):
    """Convertir le Markdown en texte brut."""
    html_content = markdown(markdown_content)  # Convertir en HTML
    soup = BeautifulSoup(html_content, "html.parser")  # Extraire le texte
    return soup.get_text()

def get_readme_content(readme_url):
    """Télécharge et retourne le contenu brut du README."""
    if not readme_url:
        return "README non disponible"

    response = requests.get(readme_url, headers=HEADERS)
    
    if response.status_code == 200:
        return markdown_to_text(response.text)  # Convertir Markdown en texte brut
    else:
        print(f"❌ Erreur {response.status_code} lors du téléchargement du README")
        return "README non disponible"

def get_readme_url(owner, repo):
    """Récupère l'URL brute du README d'un dépôt GitHub."""
    readme_url = find_readme_url(owner, repo)  # Déjà une URL de téléchargement
    
    if readme_url:
        print(f"✅ README trouvé : {readme_url}")
        return readme_url
    else:
        print(f"❌ Aucun README trouvé pour {owner}/{repo}")
        return None
    
def find_readme_url(owner, repo):
    """Cherche récursivement le fichier README dans le dépôt GitHub."""
    api_url = f"https://api.github.com/repos/{owner}/{repo}/contents"
    
    def search_readme(url):
        """Fonction récursive pour chercher un fichier README dans le dépôt."""
        response = requests.get(url, headers=HEADERS)
        
        if response.status_code != 200:
            return None
        
        files = response.json()
        
        for file in files:
            # Vérifier si c'est un README (quel que soit son format)
            if "readme" in file["name"].lower():
                return file["download_url"]
            
            # Si c'est un dossier, continuer la recherche à l'intérieur
            if file["type"] == "dir":
                sub_dir_url = file["url"]
                found_readme = search_readme(sub_dir_url)  # Appel récursif
                if found_readme:
                    return found_readme
        
        return None  # Aucun README trouvé
    
    return search_readme(api_url)  # Lancer la recherche depuis la racine du dépôt

# Récupérer l'URL du README
readme_url = get_readme_url("tobiasvl", "pico-rl")

# Télécharger et afficher le contenu
if readme_url:
    readme_content = get_readme_content(readme_url)
    print("\n🔹 Contenu du README :")
    print(readme_content)


✅ README trouvé : https://raw.githubusercontent.com/tobiasvl/pico-rl/master/README.md

🔹 Contenu du README :
A small PICO-8 roguelike.
